<b> nouvelle stratégie d'imputing.  
jusqu'ici on faisait la moyenne uniquement des joueurs que l'on connaissait dans fifa.  
ici on essaye de remplacer les joueurs manquant par le joueur moyen avant de faire la moyenne pondérée.

# Imports

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
pd.options.display.max_columns = None

# Lecture des données

In [2]:
col_stats = ['overall' ,'potential','weak_foot', 'skill_moves', 'international_reputation', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic',
       'attacking_crossing','attacking_heading_accuracy','skill_curve','skill_fk_accuracy', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power','power_jumping', 'power_stamina','mentality_aggression',
       'mentality_positioning', 'mentality_penalties','mentality_composure','goalkeeping_diving','goalkeeping_speed']


fifa15 =pd.read_csv('../data/FIFA/players_15.csv',\
    usecols=['sofifa_id']+col_stats,\
    dtype={x:np.float16 for x in col_stats})
fifa16 =pd.read_csv('../data/FIFA/players_16.csv',\
    usecols=['sofifa_id']+col_stats,\
    dtype={x:np.float16 for x in col_stats})
fifa17 =pd.read_csv('../data/FIFA/players_17.csv',\
    usecols=['sofifa_id']+col_stats,\
    dtype={x:np.float16 for x in col_stats})
fifa18 =pd.read_csv('../data/FIFA/players_18.csv',\
    usecols=['sofifa_id']+col_stats,\
    dtype={x:np.float16 for x in col_stats})
fifa19 =pd.read_csv('../data/FIFA/players_19.csv',\
    usecols=['sofifa_id']+col_stats,\
    dtype={x:np.float16 for x in col_stats})
fifa20 =pd.read_csv('../data/FIFA/players_20.csv',\
    usecols=['sofifa_id']+col_stats,\
    dtype={x:np.float16 for x in col_stats})
fifa21 =pd.read_csv('../data/FIFA/players_21.csv',\
    usecols=['sofifa_id']+col_stats,\
    dtype={x:np.float16 for x in col_stats})
fifa22 =pd.read_csv('../data/FIFA/players_22.csv',\
    usecols=['sofifa_id']+col_stats,\
    dtype={x:np.float16 for x in col_stats})

In [3]:
clubs = pd.read_csv('../data/games/clubs.csv', index_col = 0)
leagues = pd.read_csv('../data/games/leagues.csv', index_col = 0)
games = pd.read_csv('../data/games/games.csv', index_col = 0,
                   dtype = {'game_id' : np.int32, 'season' : np.int16, 'home_club_id' : np.int32, 'home_club_id' : np.int32,
                           'home_club_goals' : np.int8, 'away_club_goals' : np.int8})

players = pd.read_csv('../data/games/players.csv', index_col = 0,
                     dtype = {'player_id' : np.int32, 'player_club_id' : np.int16})

appearance = pd.read_csv('../data/games/appearance.csv', index_col = 0)
corr = pd.read_csv('../data/table_correspondance/table_correspondance_finale.csv', index_col = 0)
player_IDs = corr.player_id.unique()


In [4]:
appearance_select =\
pd.read_csv('../data/training_data/appearance_select6.csv', index_col = 0, \
            dtype = {'game_id' : np.int32, 'player_id' : np.int32,'player_club_id' : np.int16, 'minutes_played' : np.int8,\
                    'yellow_cards' : np.int8,'red_cards' : np.int8,'goals' : np.int8,'assists' : np.int8,
                    'max_time' : np.int16,'time_ratio' : np.float16, 'season' : np.int8})

appearance_known_players =\
pd.read_csv('../data/training_data/appearance_known_players6.csv', index_col = 0,\
            dtype = {'game_id' : np.int32, 'player_id' : np.int32,'player_club_id' : np.int16, 'minutes_played' : np.int8,\
                    'yellow_cards' : np.int8,'red_cards' : np.int8,'goals' : np.int8,'assists' : np.int8,
                    'max_time' : np.int16,'time_ratio' : np.float16, 'season' : np.int8})

train1 = pd.read_csv('data/training_data/train1_6.csv', index_col = 0)

In [6]:
fifa15.head(1)

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,100500000.0,550000.0,27,1987-06-24,169,67,241.0,FC Barcelona,Spain Primera Division,1.0,CF,10.0,NaN,2004-07-01,2018.0,52,Argentina,1369.0,CF,10.0,Left,3,4,5,Medium/Low,Normal (170-),Yes,NaN,"#Speedster, #Dribbler, #FK Specialist, #Acroba...","Finesse Shot, Speed Dribbler (AI), One Club Pl...",93.0,89.0,86.0,96.0,27.0,63.0,84,94,71,89,85,96,89,90,76,96,96,90,94,94,95,80,73,77,60,88,48,22,92,90,76,NaN,25,21,20,6,11,15,14,8,NaN,89+3,89+3,89+3,92+3,90+3,90+3,90+3,92+3,92+3,92+3,92+3,90+3,79+3,79+3,79+3,90+3,62+3,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png


#### colonnes des statistiques fifa à sélectionner

In [5]:
stats_col = ['overall' ,'potential','weak_foot', 'skill_moves', 'international_reputation', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic',
       'attacking_crossing','attacking_heading_accuracy','skill_curve','skill_fk_accuracy', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power','power_jumping', 'power_stamina','mentality_aggression',
       'mentality_positioning', 'mentality_penalties','mentality_composure']


GK_stats_col = ['overall' ,'potential','attacking_crossing','attacking_heading_accuracy','skill_curve','skill_fk_accuracy', 
                'movement_agility','movement_reactions', 'movement_balance', 'power_shot_power','power_jumping', 'power_stamina',
                'mentality_aggression','mentality_positioning', 'mentality_penalties','goalkeeping_diving', 
                'goalkeeping_speed']

#### Identification des quelques matchs sans GK

In [6]:

bad_game_ids = train1.loc[train1.GK < 0.8,'game_id'].unique()

# Création d'un profil moyen de joueurs d'attaque/défense/milieu/GK
On prend la moyenne des notes des joueurs de chaque poste divisée par 2.  
On suppose en effet que si un joueur n'est pas dans les fichier fifa, c'est qu'il est moins bon que la moyenne.

In [7]:
# Arrangeons un petit peu le df players
players.loc[players.player_id == 290801,'position'] = 'Defender'
players.loc[players.player_id == 290801,'sub_position'] = 'Centre-Back'
t  = players.loc[players.position == '0','sub_position']
players.loc[t.index,'position'] = t
players.drop(columns =
             ['name','pretty_name','country_of_birth','country_of_citizenship','foot','height_in_cm','market_value_in_gbp' ,'highest_market_value_in_gbp' ,'url'],
            inplace = True)
players.position = players.position.apply(lambda x: x.lower() )


In [8]:
# liste des joueurs de la table de correspondance pouvant être trouvés dans fifa22:
players22_ID = corr.loc[corr.which_fifa==22,'player_id'].values

# faisons un df ne contenant que ces joueurs
players22 = players.loc[players.player_id.isin(players22_ID),:].reset_index(drop=True).copy()

In [9]:
players22.position.unique()

array(['defender', 'midfield', 'goalkeeper', 'attack'], dtype=object)

In [10]:
GK22 = players22.loc[players22.position == 'goalkeeper',:].reset_index(drop=True).copy()
MID22 = players22.loc[players22.position == 'midfield',:].reset_index(drop=True).copy()
DEF22 = players22.loc[players22.position == 'defender',:].reset_index(drop=True).copy()
ATK22 = players22.loc[players22.position == 'attack',:].reset_index(drop=True).copy()

## GK Moyen

In [11]:
def seek_statGK22(x):
    sofID = corr.loc[(corr.player_id == x) & (corr.which_fifa == 22),'sofifa_id'].values[0]
    return fifa22.loc[fifa22.sofifa_id==sofID,GK_stats_col]

tt=GK22.player_id.progress_apply(seek_statGK22)
all_GK = pd.concat(tt.values.tolist())
####====>
avg_GK = all_GK.mean(axis=0)/2

100%|██████████| 880/880 [00:01<00:00, 649.98it/s]


## ATK, DEF, & MID

In [12]:
def seek_stat22(x):
    sofID = corr.loc[(corr.player_id == x) & (corr.which_fifa == 22),'sofifa_id'].values[0]
    return fifa22.loc[fifa22.sofifa_id==sofID,stats_col]

tt=ATK22.player_id.progress_apply(seek_stat22)
all_ATK = pd.concat(tt.values.tolist())
avg_ATK = (all_ATK.astype(np.float64).mean(axis=0)/2).astype(np.float16)


tt=DEF22.player_id.progress_apply(seek_stat22)
all_DEF = pd.concat(tt.values.tolist())
avg_DEF = (all_DEF.astype(np.float64).mean(axis=0)/2).astype(np.float16)

tt=MID22.player_id.progress_apply(seek_stat22)
all_MID = pd.concat(tt.values.tolist())
avg_MID = (all_MID.astype(np.float64).mean(axis=0)/2).astype(np.float16)

100%|██████████| 2387/2387 [00:03<00:00, 665.67it/s]


# stats des GK

In [13]:
#On commence par retirer les matchs ou il n'y a pas de GK parce que c'est bizarre

appearance_select = \
appearance_select.drop(appearance_select.loc[appearance_select.game_id.isin(bad_game_ids),:].index).reset_index(drop = True)

appearance_known_players = \
appearance_known_players.drop(appearance_known_players.loc[appearance_known_players.game_id.isin(bad_game_ids),:].index).reset_index(drop = True)

train1 = train1.drop(train1.loc[train1.game_id.isin(bad_game_ids),:].index).reset_index(drop = True)


In [14]:
# On créé un dataframe à partir de appearance_known_players avec uniquement les GKs

known_GK = appearance_known_players.loc[appearance_known_players.position == 'goalkeeper',
                                        ['game_id','player_club_id', 'time_ratio','which_fifa','player_id']].reset_index(drop = True).copy()
#On garde en tête qu'on a pu perdre des lignes voire des équipes dans des matches

In [15]:
# Test d'une ligne d'un apply:
#On extrait les stats (GK_stats_col) d'un player_id dans un dataframe fifa désigné par which_fifa 

x = known_GK.iloc[3,:]
locs = locals()
sofifa = corr.loc[(corr.player_id == x['player_id']) & (corr.which_fifa == x['which_fifa']), 'sofifa_id'].values[0]
FIFA = locs['fifa'+str(int(x['which_fifa']))]
FIFA.loc[FIFA.sofifa_id==sofifa,GK_stats_col]


,overall,potential,attacking_crossing,attacking_heading_accuracy,skill_curve,skill_fk_accuracy,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,mentality_aggression,mentality_positioning,mentality_penalties,goalkeeping_diving,goalkeeping_speed
263,80.0,84.0,25.0,25.0,25.0,25.0,42.0,79.0,37.0,28.0,76.0,45.0,30.0,25.0,35.0,81.0,49.0


In [16]:
#On extrait les stats (GK_stats_col) d'un player_id dans un dataframe fifa désigné par which_fifa 

locs = locals()
def GK_stats(x):
    sofifa = corr.loc[(corr.player_id == x['player_id']) & (corr.which_fifa == x['which_fifa']), 'sofifa_id'].values[0]
    FIFA = locs['fifa'+str(int(x['which_fifa']))]
    
    return FIFA.loc[FIFA.sofifa_id==sofifa,GK_stats_col]
    
    
    
GK_stats = known_GK.progress_apply(lambda x: GK_stats(x)
                          ,axis = 1
                         )
GK_stats = pd.concat(GK_stats.values.tolist()).reset_index(drop = True)

100%|██████████| 59459/59459 [01:42<00:00, 579.79it/s]


In [135]:
#on renomme les colonnes de GK_stats pour ajouter GK au début!
#GK_stats.rename(columns = {x:'GK_'+x for x in GK_stats.columns}, inplace = True)
GK_stats.head(1)

,GK_overall,GK_potential,GK_attacking_crossing,GK_attacking_heading_accuracy,GK_skill_curve,GK_skill_fk_accuracy,GK_movement_agility,GK_movement_reactions,GK_movement_balance,GK_power_shot_power,GK_power_jumping,GK_power_stamina,GK_mentality_aggression,GK_mentality_positioning,GK_mentality_penalties,GK_goalkeeping_diving,GK_goalkeeping_speed
0,80.0,84.0,25.0,25.0,25.0,25.0,42.0,79.0,37.0,28.0,76.0,45.0,30.0,25.0,35.0,81.0,49.0


In [20]:
# On multiplie les stats par le time_ratio (moyenne pondérée):
GK_stats = GK_stats.multiply(known_GK.time_ratio,axis = 0)

In [136]:
# On ajoute GK_stats à known_GK
known_GK = appearance_known_players.loc[appearance_known_players.position == 'goalkeeper',
                                        ['game_id','player_club_id', 'time_ratio','which_fifa','player_id']].reset_index(drop = True).copy()
                                        
known_GK = pd.concat([known_GK , GK_stats],axis = 1)

In [137]:
# On fait la somme des stats pondérées des gardiens de but par équipes et par match
known_GK = known_GK.groupby(['game_id','player_club_id'])[np.append(['time_ratio'],GK_stats.columns)].sum().reset_index()

In [138]:
# Puis on divise par la somme des time_ratios (toujours la moyenne pondérée):

known_GK.loc[:,GK_stats.columns] = known_GK.loc[:,GK_stats.columns].divide(known_GK.time_ratio, axis = 0)


In [139]:
# à ce stade on a des lignes pour les GK utilisables pour l'entrainement du modèle.
# Mais on sait que quand on n'a selectionné que les GK que l'on connait, on a perdu quelques lignes en route. 
# En l'occurence, comme il n'y a en général qu'un GK par match et par équipe, on a dû perdre des équipes. On vérifie:

team_count = known_GK.groupby('game_id')['player_club_id'].unique().reset_index()
print(team_count.loc[team_count.player_club_id.apply(len)<2,'game_id'].shape[0])
#il en manque 1910

1910


In [140]:
# On créée le fichier d'entrainement avec pour l'instant seulement les GK.
# On fera la même chose pour les ATK, DEF et MF, puis on mergera tout ça
train_GK = train1.loc[:,['game_id','player_club_id','GK']].copy()

In [141]:
train_GK = pd.merge(train_GK , known_GK.drop(columns = ['time_ratio']), \
         on = ['player_club_id','game_id'], how = 'left')


In [142]:
train_GK.loc[train_GK.GK_overall.isnull(),:].head()

,game_id,player_club_id,GK,GK_overall,GK_potential,GK_attacking_crossing,GK_attacking_heading_accuracy,GK_skill_curve,GK_skill_fk_accuracy,GK_movement_agility,GK_movement_reactions,GK_movement_balance,GK_power_shot_power,GK_power_jumping,GK_power_stamina,GK_mentality_aggression,GK_mentality_positioning,GK_mentality_penalties,GK_goalkeeping_diving,GK_goalkeeping_speed
2,2458528,338,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
628,2460301,281,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
900,2460757,24,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
918,2460766,24,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
929,2460771,24,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [143]:
train_GK.shape

(60846, 20)

In [144]:
train_GK.loc[train_GK.GK_overall.isnull(),GK_stats.columns] = avg_GK.values
display(train_GK.loc[train_GK.GK_overall.isnull(),:])
train_GK.loc[train_GK.GK_overall == avg_GK.values[0],:].head()

,game_id,player_club_id,GK,GK_overall,GK_potential,GK_attacking_crossing,GK_attacking_heading_accuracy,GK_skill_curve,GK_skill_fk_accuracy,GK_movement_agility,GK_movement_reactions,GK_movement_balance,GK_power_shot_power,GK_power_jumping,GK_power_stamina,GK_mentality_aggression,GK_mentality_positioning,GK_mentality_penalties,GK_goalkeeping_diving,GK_goalkeeping_speed


,game_id,player_club_id,GK,GK_overall,GK_potential,GK_attacking_crossing,GK_attacking_heading_accuracy,GK_skill_curve,GK_skill_fk_accuracy,GK_movement_agility,GK_movement_reactions,GK_movement_balance,GK_power_shot_power,GK_power_jumping,GK_power_stamina,GK_mentality_aggression,GK_mentality_positioning,GK_mentality_penalties,GK_goalkeeping_diving,GK_goalkeeping_speed
2,2458528,338,1.0,34.0625,36.40625,6.96875,7.058594,7.363281,6.917969,20.578125,31.125,20.46875,24.359375,28.859375,15.148438,13.109375,5.261719,9.820312,34.375,19.3125
628,2460301,281,1.0,34.0625,36.40625,6.96875,7.058594,7.363281,6.917969,20.578125,31.125,20.46875,24.359375,28.859375,15.148438,13.109375,5.261719,9.820312,34.375,19.3125
900,2460757,24,1.0,34.0625,36.40625,6.96875,7.058594,7.363281,6.917969,20.578125,31.125,20.46875,24.359375,28.859375,15.148438,13.109375,5.261719,9.820312,34.375,19.3125
918,2460766,24,1.0,34.0625,36.40625,6.96875,7.058594,7.363281,6.917969,20.578125,31.125,20.46875,24.359375,28.859375,15.148438,13.109375,5.261719,9.820312,34.375,19.3125
929,2460771,24,1.0,34.0625,36.40625,6.96875,7.058594,7.363281,6.917969,20.578125,31.125,20.46875,24.359375,28.859375,15.148438,13.109375,5.261719,9.820312,34.375,19.3125


In [146]:
train_GK.to_csv('../data/training_data/train_GK6.csv')

# stats des Attaquants

In [35]:
known_ATK = appearance_known_players.loc[appearance_known_players.position == 'attack',
                                        ['game_id','player_club_id', 'time_ratio','which_fifa','player_id']].reset_index(drop = True).copy()


In [36]:
locs = locals()
def player_stats(x):
    sofifa = corr.loc[(corr.player_id == x['player_id']) & (corr.which_fifa == x['which_fifa']), 'sofifa_id'].values[0]
    FIFA = locs['fifa'+str(int(x['which_fifa']))]
    
    return FIFA.loc[FIFA.sofifa_id==sofifa,stats_col]
    
    
    
ATK_stats = known_ATK.progress_apply(lambda x: player_stats(x)
                          ,axis = 1
                         )
ATK_stats = pd.concat(ATK_stats.values.tolist()).reset_index(drop = True)

100%|██████████| 224873/224873 [06:43<00:00, 557.60it/s]


In [37]:
ATK_stats.rename(columns = {x:'ATK_'+x for x in ATK_stats.columns}, inplace = True)
ATK_stats.head(1)

,ATK_overall,ATK_potential,ATK_weak_foot,ATK_skill_moves,ATK_international_reputation,ATK_pace,ATK_shooting,ATK_passing,ATK_dribbling,ATK_defending,ATK_physic,ATK_attacking_crossing,ATK_attacking_heading_accuracy,ATK_skill_curve,ATK_skill_fk_accuracy,ATK_movement_agility,ATK_movement_reactions,ATK_movement_balance,ATK_power_shot_power,ATK_power_jumping,ATK_power_stamina,ATK_mentality_aggression,ATK_mentality_positioning,ATK_mentality_penalties,ATK_mentality_composure
0,56.0,63.0,3.0,3.0,1.0,52.0,55.0,40.0,55.0,30.0,64.0,27.0,58.0,33.0,34.0,55.0,54.0,45.0,60.0,68.0,65.0,51.0,48.0,37.0,NaN


In [73]:
appearance_ATK = appearance_select.loc[appearance_select.position == 'attack',
                                        ['game_id','player_club_id', 'time_ratio','player_id']].reset_index(drop = True).copy()

known_ATK = appearance_known_players.loc[appearance_known_players.position == 'attack',
                                        ['game_id','player_club_id','player_id']].reset_index(drop = True).copy()

known_ATK = pd.concat([known_ATK , ATK_stats],axis = 1)

#all_ATK existe déjà comme nom de variable mais peu importe on réécrit par dessus
all_ATK = pd.merge(appearance_ATK, known_ATK, how = 'left', on = ['game_id','player_club_id','player_id'])

In [74]:
# On remplace les lignes manquantes par l'attaquant moyen
all_ATK.loc[all_ATK.ATK_overall.isnull(),ATK_stats.columns] = avg_ATK.values
all_ATK.loc[all_ATK.ATK_overall.isnull(),:]

all_ATK.loc[:,ATK_stats.columns] = all_ATK.loc[:,ATK_stats.columns].multiply(all_ATK.time_ratio, axis = 0)

all_ATK = all_ATK.groupby(['game_id','player_club_id'])[np.append(['time_ratio'],ATK_stats.columns)].sum().reset_index()
all_ATK.loc[:,ATK_stats.columns] = all_ATK.loc[:,ATK_stats.columns].divide(all_ATK.time_ratio, axis = 0)

In [80]:
# On créée le fichier d'entrainement avec les ATK:
train_ATK = train1.loc[:,['game_id','player_club_id','attack']].copy()
train_ATK = pd.merge(train_ATK , all_ATK.drop(columns = ['time_ratio']), \
         on = ['player_club_id','game_id'], how = 'left')

#S'il n'y a pas d'attaquants, alors attack=0 et on met les stats à 0
train_ATK.loc[train_ATK.attack == 0,ATK_stats.columns]=0
train_ATK.loc[train_ATK.attack == 0,ATK_stats.columns].head(1)

In [86]:
train_ATK.to_csv('../data/training_data/train_ATK6_bis.csv')
#train_ATK.to_csv('../experimentation/data/training_data/train_ATK6.csv')

# Stats des défenseurs

In [87]:
known_DEF = appearance_known_players.loc[appearance_known_players.position == 'defender',
                                        ['game_id','player_club_id', 'time_ratio','which_fifa','player_id']].reset_index(drop = True).copy()


In [88]:
DEF_stats = known_DEF.progress_apply(lambda x: player_stats(x)
                          ,axis = 1
                         )
DEF_stats = pd.concat(DEF_stats.values.tolist()).reset_index(drop = True)

100%|██████████| 266843/266843 [07:25<00:00, 599.22it/s]


In [89]:
DEF_stats.rename(columns = {x:'DEF_'+x for x in DEF_stats.columns}, inplace = True)
DEF_stats.head(1)

,DEF_overall,DEF_potential,DEF_weak_foot,DEF_skill_moves,DEF_international_reputation,DEF_pace,DEF_shooting,DEF_passing,DEF_dribbling,DEF_defending,DEF_physic,DEF_attacking_crossing,DEF_attacking_heading_accuracy,DEF_skill_curve,DEF_skill_fk_accuracy,DEF_movement_agility,DEF_movement_reactions,DEF_movement_balance,DEF_power_shot_power,DEF_power_jumping,DEF_power_stamina,DEF_mentality_aggression,DEF_mentality_positioning,DEF_mentality_penalties,DEF_mentality_composure
0,68.0,74.0,2.0,2.0,1.0,72.0,62.0,66.0,67.0,66.0,68.0,68.0,61.0,65.0,48.0,68.0,69.0,68.0,65.0,67.0,75.0,74.0,65.0,50.0,NaN


In [98]:
appearance_DEF = appearance_select.loc[appearance_select.position == 'defender',
                                        ['game_id','player_club_id', 'time_ratio','player_id']].reset_index(drop = True).copy()

known_DEF = appearance_known_players.loc[appearance_known_players.position == 'defender',
                                        ['game_id','player_club_id','player_id']].reset_index(drop = True).copy()

known_DEF = pd.concat([known_DEF , DEF_stats],axis = 1)

all_DEF = pd.merge(appearance_DEF, known_DEF, how = 'left', on = ['game_id','player_club_id','player_id'])

In [99]:
all_DEF.loc[all_DEF.DEF_overall.isnull(),DEF_stats.columns] = avg_DEF.values
all_DEF.loc[all_DEF.DEF_overall.isnull(),:]

all_DEF.loc[:,DEF_stats.columns] = all_DEF.loc[:,DEF_stats.columns].multiply(all_DEF.time_ratio, axis = 0)
all_DEF = all_DEF.groupby(['game_id','player_club_id'])[np.append(['time_ratio'],DEF_stats.columns)].sum().reset_index()
all_DEF.loc[:,DEF_stats.columns] = all_DEF.loc[:,DEF_stats.columns].divide(all_DEF.time_ratio, axis = 0)

In [103]:
# On créée le fichier d'entrainement avec les ATK:
train_DEF = train1.loc[:,['game_id','player_club_id','defense']].copy()
train_DEF = pd.merge(train_DEF , all_DEF.drop(columns = ['time_ratio']), \
         on = ['player_club_id','game_id'], how = 'left')

#S'il n'y a pas d'attaquants, alors attack=0 et on met les stats à 0
train_DEF.loc[train_DEF.defense == 0,DEF_stats.columns]=0
train_DEF.loc[train_DEF.defense == 0,DEF_stats.columns].head(1)

,DEF_overall,DEF_potential,DEF_weak_foot,DEF_skill_moves,DEF_international_reputation,DEF_pace,DEF_shooting,DEF_passing,DEF_dribbling,DEF_defending,DEF_physic,DEF_attacking_crossing,DEF_attacking_heading_accuracy,DEF_skill_curve,DEF_skill_fk_accuracy,DEF_movement_agility,DEF_movement_reactions,DEF_movement_balance,DEF_power_shot_power,DEF_power_jumping,DEF_power_stamina,DEF_mentality_aggression,DEF_mentality_positioning,DEF_mentality_penalties,DEF_mentality_composure
52863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [105]:
train_DEF.to_csv('../experimentation/data/training_data/train_DEF6_bis.csv')

# Stats des milieux de terrain

In [106]:
known_MF = appearance_known_players.loc[appearance_known_players.position == 'midfield',
                                        ['game_id','player_club_id', 'time_ratio','which_fifa','player_id']].reset_index(drop = True).copy()


In [107]:
MF_stats = known_MF.progress_apply(lambda x: player_stats(x)
                          ,axis = 1
                         )
MF_stats = pd.concat(MF_stats.values.tolist()).reset_index(drop = True)

100%|██████████| 246963/246963 [07:07<00:00, 578.10it/s]


In [111]:
MF_stats.rename(columns = {x:'MF_'+x for x in MF_stats.columns}, inplace = True)
MF_stats.head(1)

,MF_overall,MF_potential,MF_weak_foot,MF_skill_moves,MF_international_reputation,MF_pace,MF_shooting,MF_passing,MF_dribbling,MF_defending,MF_physic,MF_attacking_crossing,MF_attacking_heading_accuracy,MF_skill_curve,MF_skill_fk_accuracy,MF_movement_agility,MF_movement_reactions,MF_movement_balance,MF_power_shot_power,MF_power_jumping,MF_power_stamina,MF_mentality_aggression,MF_mentality_positioning,MF_mentality_penalties,MF_mentality_composure
0,82.0,82.0,3.0,3.0,3.0,66.0,81.0,84.0,79.0,52.0,70.0,86.0,60.0,87.0,89.0,72.0,78.0,61.0,82.0,70.0,86.0,57.0,80.0,80.0,81.0


In [112]:
appearance_MF = appearance_select.loc[appearance_select.position == 'midfield',
                                        ['game_id','player_club_id', 'time_ratio','player_id']].reset_index(drop = True).copy()

known_MF = appearance_known_players.loc[appearance_known_players.position == 'midfield',
                                        ['game_id','player_club_id','player_id']].reset_index(drop = True).copy()

known_MF = pd.concat([known_MF , MF_stats],axis = 1)

all_MF = pd.merge(appearance_MF, known_MF, how = 'left', on = ['game_id','player_club_id','player_id'])

In [113]:
all_MF.loc[all_MF.MF_overall.isnull(),MF_stats.columns] = avg_MID.values
all_MF.loc[all_MF.MF_overall.isnull(),:]

all_MF.loc[:,MF_stats.columns] = all_MF.loc[:,MF_stats.columns].multiply(all_MF.time_ratio, axis = 0)
all_MF = all_MF.groupby(['game_id','player_club_id'])[np.append(['time_ratio'],MF_stats.columns)].sum().reset_index()
all_MF.loc[:,MF_stats.columns] = all_MF.loc[:,MF_stats.columns].divide(all_MF.time_ratio, axis = 0)

In [114]:
# On créée le fichier d'entrainement avec les ATK:
train_MF = train1.loc[:,['game_id','player_club_id','midfield']].copy()
train_MF = pd.merge(train_MF , all_MF.drop(columns = ['time_ratio']), \
         on = ['player_club_id','game_id'], how = 'left')

#S'il n'y a pas de milieu de terrain on met les stats à 0
train_MF.loc[train_MF.midfield == 0,MF_stats.columns]=0
train_MF.loc[train_MF.midfield == 0,MF_stats.columns].head(1)

,MF_overall,MF_potential,MF_weak_foot,MF_skill_moves,MF_international_reputation,MF_pace,MF_shooting,MF_passing,MF_dribbling,MF_defending,MF_physic,MF_attacking_crossing,MF_attacking_heading_accuracy,MF_skill_curve,MF_skill_fk_accuracy,MF_movement_agility,MF_movement_reactions,MF_movement_balance,MF_power_shot_power,MF_power_jumping,MF_power_stamina,MF_mentality_aggression,MF_mentality_positioning,MF_mentality_penalties,MF_mentality_composure
9914,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
###############################################################################################################################################################################

###############################################################################################################################################################################

###############################################################################################################################################################################

In [115]:
train_MF.to_csv('../experimentation/data/training_data/train_MF6_bis.csv')

# Création du train dataset

In [147]:
train = pd.merge(train_GK , train_ATK, \
         on = ['player_club_id','game_id'], how = 'inner')


In [148]:
train = pd.merge(train , train_DEF, \
         on = ['player_club_id','game_id'], how = 'inner')
train = pd.merge(train , train_MF, \
         on = ['player_club_id','game_id'], how = 'inner')
train.head(1)

,game_id,player_club_id,GK,GK_overall,GK_potential,GK_attacking_crossing,GK_attacking_heading_accuracy,GK_skill_curve,GK_skill_fk_accuracy,GK_movement_agility,GK_movement_reactions,GK_movement_balance,GK_power_shot_power,GK_power_jumping,GK_power_stamina,GK_mentality_aggression,GK_mentality_positioning,GK_mentality_penalties,GK_goalkeeping_diving,GK_goalkeeping_speed,attack,ATK_overall,ATK_potential,ATK_weak_foot,ATK_skill_moves,ATK_international_reputation,ATK_pace,ATK_shooting,ATK_passing,ATK_dribbling,ATK_defending,ATK_physic,ATK_attacking_crossing,ATK_attacking_heading_accuracy,ATK_skill_curve,ATK_skill_fk_accuracy,ATK_movement_agility,ATK_movement_reactions,ATK_movement_balance,ATK_power_shot_power,ATK_power_jumping,ATK_power_stamina,ATK_mentality_aggression,ATK_mentality_positioning,ATK_mentality_penalties,ATK_mentality_composure,defense,DEF_overall,DEF_potential,DEF_weak_foot,DEF_skill_moves,DEF_international_reputation,DEF_pace,DEF_shooting,DEF_passing,DEF_dribbling,DEF_defending,DEF_physic,DEF_attacking_crossing,DEF_attacking_heading_accuracy,DEF_skill_curve,DEF_skill_fk_accuracy,DEF_movement_agility,DEF_movement_reactions,DEF_movement_balance,DEF_power_shot_power,DEF_power_jumping,DEF_power_stamina,DEF_mentality_aggression,DEF_mentality_positioning,DEF_mentality_penalties,DEF_mentality_composure,midfield,MF_overall,MF_potential,MF_weak_foot,MF_skill_moves,MF_international_reputation,MF_pace,MF_shooting,MF_passing,MF_dribbling,MF_defending,MF_physic,MF_attacking_crossing,MF_attacking_heading_accuracy,MF_skill_curve,MF_skill_fk_accuracy,MF_movement_agility,MF_movement_reactions,MF_movement_balance,MF_power_shot_power,MF_power_jumping,MF_power_stamina,MF_mentality_aggression,MF_mentality_positioning,MF_mentality_penalties,MF_mentality_composure
0,2457642,610,1.0,77.0,77.0,25.0,25.0,25.0,25.0,68.0,79.0,58.0,23.0,82.0,42.0,34.0,25.0,29.0,82.0,58.0,3.000244,71.0,82.4375,3.21875,3.0,1.0,75.8125,62.25,65.3125,72.4375,32.90625,58.125,63.40625,62.34375,67.5625,55.28125,75.3125,66.25,66.625,68.8125,59.25,63.71875,45.53125,63.15625,55.96875,0.0,4.43335,72.125,76.0,3.126953,2.224609,1.225586,72.625,43.78125,61.1875,67.4375,71.1875,70.4375,55.5,72.5625,48.5,31.375,66.5,70.1875,64.9375,61.84375,78.3125,73.0,70.0625,55.3125,40.0,0.0,2.566895,75.1875,78.8125,3.609375,2.611328,1.389648,73.5,71.375,71.625,76.125,54.78125,65.5625,62.34375,60.96875,68.9375,59.8125,75.3125,75.25,73.5,73.5625,63.375,78.8125,59.28125,74.6875,67.125,0.0


In [116]:
# il faut réussir à diviser le dataset en home et away, puis faire un join.

In [149]:
#repérons les club_id qui n'existent pas
club_IDs = clubs.club_id.unique()
train.loc[~train.player_club_id.isin(club_IDs),'player_club_id'].unique()

array([-25110, -25814, -24335, -24305,  -4587, -24262, -15834])

In [297]:
tt = train.loc[train.player_club_id == -15834,'game_id'].values[2]
games.loc[games.game_id==tt,:]

,game_id,competition_code,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,away_club_position,stadium,attendance,referee,url
40682,3602415,RU1,2021,3. Matchday,2021-08-07,232,49702,1,2,11.0,5.0,Otkrytie Bank Arena,2849.0,Aleksey Sukhoy,https://www.transfermarkt.co.uk/spartak-moscow...


1. -25110 => 40426
2. -25814 => 39722
3. -24335 => 41201
4. -24305 => 41231
5. -24262 => 41274
6. -4587 => 60949  
7. -15834 => 49702

In [150]:
#on a repéré quelle mauvaise notation correspondait à qui, maintenant changeons les valeurs:
train.loc[train.player_club_id == -25110,'player_club_id'] = 40426
train.loc[train.player_club_id == -25814,'player_club_id'] = 39722
train.loc[train.player_club_id == -24335,'player_club_id'] = 41201
train.loc[train.player_club_id == -24305,'player_club_id'] = 41231
train.loc[train.player_club_id == -24262,'player_club_id'] = 41274
train.loc[train.player_club_id == -4587,'player_club_id'] = 60949
train.loc[train.player_club_id == -15834,'player_club_id'] = 49702

In [151]:
#maintenant c'est bon!
train.loc[~train.player_club_id.isin(club_IDs),'player_club_id'].unique()

array([], dtype=int64)

In [152]:
def HomeOrAway(x):
    if games.loc[games.game_id == x['game_id'],'home_club_id'].values[0] == x['player_club_id']:
        return 'Home'
    elif games.loc[games.game_id == x['game_id'],'away_club_id'].values[0] == x['player_club_id']:
        return 'Away'
    else:
        return 'problem'    

In [153]:
train['HomeOrAway']=None
train['HomeOrAway'] = train.progress_apply(HomeOrAway, axis = 1)


100%|██████████| 60846/60846 [00:29<00:00, 2030.48it/s]


In [154]:
train_home = train.loc[train.HomeOrAway == 'Home',:].drop(columns = ['HomeOrAway','player_club_id']).copy()
train_away = train.loc[train.HomeOrAway == 'Away',:].drop(columns = ['HomeOrAway','player_club_id']).copy()

train_home.rename(columns={x:'H_'+x for x in train_home.columns[1:]}, inplace = True)
train_away.rename(columns={x:'A_'+x for x in train_away.columns[1:]}, inplace = True)

train_final = pd.merge(train_home , train_away, on = ['game_id'], how = 'inner')
train_final.head()

,game_id,H_GK,H_GK_overall,H_GK_potential,H_GK_attacking_crossing,H_GK_attacking_heading_accuracy,H_GK_skill_curve,H_GK_skill_fk_accuracy,H_GK_movement_agility,H_GK_movement_reactions,H_GK_movement_balance,H_GK_power_shot_power,H_GK_power_jumping,H_GK_power_stamina,H_GK_mentality_aggression,H_GK_mentality_positioning,H_GK_mentality_penalties,H_GK_goalkeeping_diving,H_GK_goalkeeping_speed,H_attack,H_ATK_overall,H_ATK_potential,H_ATK_weak_foot,H_ATK_skill_moves,H_ATK_international_reputation,H_ATK_pace,H_ATK_shooting,H_ATK_passing,H_ATK_dribbling,H_ATK_defending,H_ATK_physic,H_ATK_attacking_crossing,H_ATK_attacking_heading_accuracy,H_ATK_skill_curve,H_ATK_skill_fk_accuracy,H_ATK_movement_agility,H_ATK_movement_reactions,H_ATK_movement_balance,H_ATK_power_shot_power,H_ATK_power_jumping,H_ATK_power_stamina,H_ATK_mentality_aggression,H_ATK_mentality_positioning,H_ATK_mentality_penalties,H_ATK_mentality_composure,H_defense,H_DEF_overall,H_DEF_potential,H_DEF_weak_foot,H_DEF_skill_moves,H_DEF_international_reputation,H_DEF_pace,H_DEF_shooting,H_DEF_passing,H_DEF_dribbling,H_DEF_defending,H_DEF_physic,H_DEF_attacking_crossing,H_DEF_attacking_heading_accuracy,H_DEF_skill_curve,H_DEF_skill_fk_accuracy,H_DEF_movement_agility,H_DEF_movement_reactions,H_DEF_movement_balance,H_DEF_power_shot_power,H_DEF_power_jumping,H_DEF_power_stamina,H_DEF_mentality_aggression,H_DEF_mentality_positioning,H_DEF_mentality_penalties,H_DEF_mentality_composure,H_midfield,H_MF_overall,H_MF_potential,H_MF_weak_foot,H_MF_skill_moves,H_MF_international_reputation,H_MF_pace,H_MF_shooting,H_MF_passing,H_MF_dribbling,H_MF_defending,H_MF_physic,H_MF_attacking_crossing,H_MF_attacking_heading_accuracy,H_MF_skill_curve,H_MF_skill_fk_accuracy,H_MF_movement_agility,H_MF_movement_reactions,H_MF_movement_balance,H_MF_power_shot_power,H_MF_power_jumping,H_MF_power_stamina,H_MF_mentality_aggression,H_MF_mentality_positioning,H_MF_mentality_penalties,H_MF_mentality_composure,A_GK,A_GK_overall,A_GK_potential,A_GK_attacking_crossing,A_GK_attacking_heading_accuracy,A_GK_skill_curve,A_GK_skill_fk_accuracy,A_GK_movement_agility,A_GK_movement_reactions,A_GK_movement_balance,A_GK_power_shot_power,A_GK_power_jumping,A_GK_power_stamina,A_GK_mentality_aggression,A_GK_mentality_positioning,A_GK_mentality_penalties,A_GK_goalkeeping_diving,A_GK_goalkeeping_speed,A_attack,A_ATK_overall,A_ATK_potential,A_ATK_weak_foot,A_ATK_skill_moves,A_ATK_international_reputation,A_ATK_pace,A_ATK_shooting,A_ATK_passing,A_ATK_dribbling,A_ATK_defending,A_ATK_physic,A_ATK_attacking_crossing,A_ATK_attacking_heading_accuracy,A_ATK_skill_curve,A_ATK_skill_fk_accuracy,A_ATK_movement_agility,A_ATK_movement_reactions,A_ATK_movement_balance,A_ATK_power_shot_power,A_ATK_power_jumping,A_ATK_power_stamina,A_ATK_mentality_aggression,A_ATK_mentality_positioning,A_ATK_mentality_penalties,A_ATK_mentality_composure,A_defense,A_DEF_overall,A_DEF_potential,A_DEF_weak_foot,A_DEF_skill_moves,A_DEF_international_reputation,A_DEF_pace,A_DEF_shooting,A_DEF_passing,A_DEF_dribbling,A_DEF_defending,A_DEF_physic,A_DEF_attacking_crossing,A_DEF_attacking_heading_accuracy,A_DEF_skill_curve,A_DEF_skill_fk_accuracy,A_DEF_movement_agility,A_DEF_movement_reactions,A_DEF_movement_balance,A_DEF_power_shot_power,A_DEF_power_jumping,A_DEF_power_stamina,A_DEF_mentality_aggression,A_DEF_mentality_positioning,A_DEF_mentality_penalties,A_DEF_mentality_composure,A_midfield,A_MF_overall,A_MF_potential,A_MF_weak_foot,A_MF_skill_moves,A_MF_international_reputation,A_MF_pace,A_MF_shooting,A_MF_passing,A_MF_dribbling,A_MF_defending,A_MF_physic,A_MF_attacking_crossing,A_MF_attacking_heading_accuracy,A_MF_skill_curve,A_MF_skill_fk_accuracy,A_MF_movement_agility,A_MF_movement_reactions,A_MF_movement_balance,A_MF_power_shot_power,A_MF_power_jumping,A_MF_power_stamina,A_MF_mentality_aggression,A_MF_mentality_positioning,A_MF_mentality_penalties,A_MF_mentality_composure
0,2457642,1.0,70.0,70.0,25.0,25.0,25.0,25.0,36.0,65.0,41.0,27.0,45.0,40.0,30.0,25.0,45.0,68

In [155]:
x = train_final.game_id[4]
games.loc[games.game_id==x,'home_club_goals'].values[0]-games.loc[games.game_id==x,'away_club_goals'].values[0]

-1

In [156]:
train_final['Home_result'] = train_final.game_id.progress_apply(lambda x:
                                                                games.loc[games.game_id==x,'home_club_goals'].values[0]-games.loc[games.game_id==x,'away_club_goals'].values[0]
                                                               )

100%|██████████| 30423/30423 [00:17<00:00, 1758.68it/s]


In [127]:
train_final.head(3)

,game_id,H_GK,H_GK_overall,H_GK_potential,H_GK_attacking_crossing,H_GK_attacking_heading_accuracy,H_GK_skill_curve,H_GK_skill_fk_accuracy,H_GK_movement_agility,H_GK_movement_reactions,H_GK_movement_balance,H_GK_power_shot_power,H_GK_power_jumping,H_GK_power_stamina,H_GK_mentality_aggression,H_GK_mentality_positioning,H_GK_mentality_penalties,H_GK_goalkeeping_diving,H_GK_goalkeeping_speed,H_GK_GK_overall,H_GK_GK_potential,H_GK_GK_attacking_crossing,H_GK_GK_attacking_heading_accuracy,H_GK_GK_skill_curve,H_GK_GK_skill_fk_accuracy,H_GK_GK_movement_agility,H_GK_GK_movement_reactions,H_GK_GK_movement_balance,H_GK_GK_power_shot_power,H_GK_GK_power_jumping,H_GK_GK_power_stamina,H_GK_GK_mentality_aggression,H_GK_GK_mentality_positioning,H_GK_GK_mentality_penalties,H_GK_GK_goalkeeping_diving,H_GK_GK_goalkeeping_speed,H_attack,H_ATK_overall,H_ATK_potential,H_ATK_weak_foot,H_ATK_skill_moves,H_ATK_international_reputation,H_ATK_pace,H_ATK_shooting,H_ATK_passing,H_ATK_dribbling,H_ATK_defending,H_ATK_physic,H_ATK_attacking_crossing,H_ATK_attacking_heading_accuracy,H_ATK_skill_curve,H_ATK_skill_fk_accuracy,H_ATK_movement_agility,H_ATK_movement_reactions,H_ATK_movement_balance,H_ATK_power_shot_power,H_ATK_power_jumping,H_ATK_power_stamina,H_ATK_mentality_aggression,H_ATK_mentality_positioning,H_ATK_mentality_penalties,H_ATK_mentality_composure,H_defense,H_DEF_overall,H_DEF_potential,H_DEF_weak_foot,H_DEF_skill_moves,H_DEF_international_reputation,H_DEF_pace,H_DEF_shooting,H_DEF_passing,H_DEF_dribbling,H_DEF_defending,H_DEF_physic,H_DEF_attacking_crossing,H_DEF_attacking_heading_accuracy,H_DEF_skill_curve,H_DEF_skill_fk_accuracy,H_DEF_movement_agility,H_DEF_movement_reactions,H_DEF_movement_balance,H_DEF_power_shot_power,H_DEF_power_jumping,H_DEF_power_stamina,H_DEF_mentality_aggression,H_DEF_mentality_positioning,H_DEF_mentality_penalties,H_DEF_mentality_composure,H_midfield,H_MF_overall,H_MF_potential,H_MF_weak_foot,H_MF_skill_moves,H_MF_international_reputation,H_MF_pace,H_MF_shooting,H_MF_passing,H_MF_dribbling,H_MF_defending,H_MF_physic,H_MF_attacking_crossing,H_MF_attacking_heading_accuracy,H_MF_skill_curve,H_MF_skill_fk_accuracy,H_MF_movement_agility,H_MF_movement_reactions,H_MF_movement_balance,H_MF_power_shot_power,H_MF_power_jumping,H_MF_power_stamina,H_MF_mentality_aggression,H_MF_mentality_positioning,H_MF_mentality_penalties,H_MF_mentality_composure,A_GK,A_GK_overall,A_GK_potential,A_GK_attacking_crossing,A_GK_attacking_heading_accuracy,A_GK_skill_curve,A_GK_skill_fk_accuracy,A_GK_movement_agility,A_GK_movement_reactions,A_GK_movement_balance,A_GK_power_shot_power,A_GK_power_jumping,A_GK_power_stamina,A_GK_mentality_aggression,A_GK_mentality_positioning,A_GK_mentality_penalties,A_GK_goalkeeping_diving,A_GK_goalkeeping_speed,A_GK_GK_overall,A_GK_GK_potential,A_GK_GK_attacking_crossing,A_GK_GK_attacking_heading_accuracy,A_GK_GK_skill_curve,A_GK_GK_skill_fk_accuracy,A_GK_GK_movement_agility,A_GK_GK_movement_reactions,A_GK_GK_movement_balance,A_GK_GK_power_shot_power,A_GK_GK_power_jumping,A_GK_GK_power_stamina,A_GK_GK_mentality_aggression,A_GK_GK_mentality_positioning,A_GK_GK_mentality_penalties,A_GK_GK_goalkeeping_diving,A_GK_GK_goalkeeping_speed,A_attack,A_ATK_overall,A_ATK_potential,A_ATK_weak_foot,A_ATK_skill_moves,A_ATK_international_reputation,A_ATK_pace,A_ATK_shooting,A_ATK_passing,A_ATK_dribbling,A_ATK_defending,A_ATK_physic,A_ATK_attacking_crossing,A_ATK_attacking_heading_accuracy,A_ATK_skill_curve,A_ATK_skill_fk_accuracy,A_ATK_movement_agility,A_ATK_movement_reactions,A_ATK_movement_balance,A_ATK_power_shot_power,A_ATK_power_jumping,A_ATK_power_stamina,A_ATK_mentality_aggression,A_ATK_mentality_positioning,A_ATK_mentality_penalties,A_ATK_mentality_composure,A_defense,A_DEF_overall,A_DEF_potential,A_DEF_weak_foot,A_DEF_skill_moves,A_DEF_international_reputation,A_DEF_pace,A_DEF_shooting,A_DEF_passing,A_DEF_dribbling,A_DEF_defending,A_DEF_physic,A_DEF_attacking_crossing,A_DEF_attacking_heading_accuracy,A_DEF_skill_curve,A_DEF_s

In [157]:
train_final = train_final.astype(dtype={x: np.float16 for x in train_final.columns[1:-1]})
train_final.to_csv('../experimentation/data/training_data/train_final_6_bis.csv')